In [1]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

If you get an error saying something like "sqlalchemy not installed", you should add a code cell and write `pip install sqlalchemy`. Then, rerun the above cell (you might also need to install pymysql the same way)

In [2]:
password = '..'
db_name = '...'

In [3]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)

In [4]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [49]:
guests = pd.read_csv('Project 2 Launch  - Guests.csv')
payments = pd.read_csv('Project 2 Launch  - Payments.csv')
reservations = pd.read_csv('Project 2 Launch  - Reservations.csv')
rooms = pd.read_csv('Project 2 Launch  - Rooms.csv')

In [52]:
guests.columns = guests.columns.str.strip()
reservations.columns = reservations.columns.str.strip()

In [53]:
guests.to_sql(name='guests', con=connection, if_exists='replace', index=False)
payments.to_sql(name='payments', con=connection, if_exists='replace', index=False)
reservations.to_sql(name='reservations', con=connection, if_exists='replace', index=False)
rooms.to_sql(name='rooms', con=connection, if_exists='replace', index=False)

20

Which rooms have the highest occupancy rates? (Rooms<->Reservation)?

1. Calculate Total Booked Days for Each Room by subtracting the check-in date from the check-out date.
2. Calculate the Total Possible Days by subtracting the check-in date from the check-out date
3. Compute Occupancy Rate = total booked days/total possible days in the date range.

In [67]:
query = """
SELECT 
    rooms.ID AS roomnum,
    rooms.Type AS type,
    rooms.Capacity AS capacity,
    rooms.Price AS price,
    SUM(DATEDIFF(STR_TO_DATE(res.Checkout_Date, '%%m/%%d/%%y'), STR_TO_DATE(res.Checkin_Date, '%%m/%%d/%%y'))) AS TotalBookedDays,
    COUNT(res.Reservation_ID) AS TotalReservations,
    (SUM(DATEDIFF(STR_TO_DATE(res.Checkout_Date, '%%m/%%d/%%y'), STR_TO_DATE(res.Checkin_Date, '%%m/%%d/%%y'))) / 
     (DATEDIFF((SELECT MAX(STR_TO_DATE(res2.Checkout_Date, '%%m/%%d/%%y')) FROM reservations AS res2), 
               (SELECT MIN(STR_TO_DATE(res2.Checkin_Date, '%%m/%%d/%%y')) FROM reservations AS res2)))) * 100 AS OccupancyRate
FROM rooms
LEFT JOIN reservations AS res ON rooms.ID = res.Room_Number
GROUP BY roomnum, type, capacity, price
ORDER BY OccupancyRate DESC;
"""

In [68]:
result_df = pd.read_sql(query, con=connection)

In [82]:
result_df

,roomnum,type,capacity,price,TotalBookedDays,TotalReservations,OccupancyRate
0,8,Double,2,150,17.0,2,94.4444
1,20,Double,2,150,16.0,2,88.8889
2,3,Suite,4,300,15.0,2,83.3333
3,6,Suite,4,300,13.0,2,72.2222
4,10,Single,1,100,13.0,2,72.2222
5,19,Single,1,100,12.0,2,66.6667
6,1,Single,1,100,9.0,2,50.0000
7,4,Single,1,100,9.0,1,50.0000
8,13,Single,1,100,9.0,2,50.0000
9,14,Double,2,150,9.0,1,50.0000


In [80]:
import time
start_time = time.time()
result_df1 = pd.read_sql(query, con=connection)
end_time = time.time()
execution_time = end_time - start_time
print(f"MySQL query execution time: {execution_time} seconds")

MySQL query execution time: 0.0032050609588623047 seconds


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar(range(len(result_df)), result_df['OccupancyRate'], color='skyblue')
plt.xlabel('Room Number')
plt.ylabel('Occupancy Rate (%)')
plt.title('Occupancy Rate by Room Number')
plt.xticks(range(len(result_df)), result_df['roomnum'])
plt.grid(axis='y')
plt.show()